In [ ]:
# https://en.wikipedia.org/wiki/Low-pass_filter

# https://controlsystemsacademy.com/

%matplotlib inline
import collections

import matplotlib.pyplot as plt

Prenons un filtre du premier ordre de gain null, que l'on écrit en notation de laplace:

$$ G(s) = \frac{1}{1 + T_cs} $$

Avec :

$$ \frac{1}{Tc} = 2 \pi f_c = \omega_c $$

La transformée bilineaire est l'approximation au premier ordre de la definition de la transformée en $z$

\begin{align}
z &= e^{sT} \\
s &= \frac{1}{T} \ln(z)  \\
  &= \frac{2}{T} \left[\frac{z-1}{z+1} + \frac{1}{3} \left( \frac{z-1}{z+1} \right)^3  + \frac{1}{5} \left( \frac{z-1}{z+1} \right)^5  + \frac{1}{7} \left( \frac{z-1}{z+1} \right)^7 + \cdots \right] \\
  &\approx  \frac{2}{T} \frac{z - 1}{z + 1} \\
  &=  \frac{2}{T} \frac{1 - z^{-1}}{1 + z^{-1}}
\end{align}

In [ ]:
import sympy

T, Tc, z1 = sympy.symbols('T T_c z^-1') # la fréquence d'échantillonnage
s = (2 / T)*(1 - z1)/(1 + z1)
G = 1 / (1 + Tc *s)
G

In [ ]:
G.expand().cancel()

In [ ]:
A = (T - 2*Tc) / (T + 2*Tc)
B = T / (T + 2*Tc)
H = B * ( (1 + z1) / (1 + A*z1) )
H.expand().simplify()

In [ ]:
x, y = sympy.symbols('x y')
Gn, Gd = sympy.fraction(G.expand().simplify())
P0 = (y * Gd - x * Gn).expand()
P0

In [ ]:
P1 = (T*x*z1 + T*x + 2*Tc*y*z1 - T*y*z1) - (T*y+2*Tc*y)

In [ ]:
P1 = (T*x*(1+z1) + (2*Tc - T)*y*z1) - ((T+2*Tc)*y)

In [ ]:
P1 = (T*x*(1+z1) + (2*Tc - T)*y*z1) / (T+2*Tc) - y

In [ ]:
P1 = (T*x*z1 + T*x + (2*Tc - T)*y*z1) / (T+2*Tc) - y

In [ ]:
(P0 + (T+2*Tc) * P1).expand().simplify()

In [ ]:
class LowPass_FirstOrder_Bilinear() :
    def __init__(self, time_constant=10.0, period=1.0, init={'x':0.0, 'y':0.0}) :
        self.Tc = time_constant
        self.dt = period

        self.h = collections.defaultdict(list)
        for k, v in init.items() :
            self.h[k].append(v)
        
    def run(self, x) :
        x_prev = self.h['x'][-1]
        y_prev = self.h['y'][-1]
        
        y = (self.dt*x_prev + self.dt*x + (2*self.Tc - self.dt)*y_prev) / (self.dt + 2*self.Tc)

        self.h['x'].append(x)
        self.h['y'].append(y)

In [ ]:
# ça c'est en fait un zero degrees (blocker) filter
# plus de détail ici : https://controlsystemsacademy.com/0020/0020.html

class LowPass_SimpleRecursive() :
    def __init__(self, time_constant=10.0, period=1.0, init={'x':0.0, 'y':0.0}) :
        self.Tc = time_constant
        self.dt = period

        self.k = 0.09

        self.h = collections.defaultdict(list)
        for k, v in init.items() :
            self.h[k].append(v)
        
    def run(self, x) :
        x_prev = self.h['x'][-1]
        y_prev = self.h['y'][-1]
        
        y = self.k*x + (1-self.k)*y_prev

        self.h['x'].append(x)
        self.h['y'].append(y)

In [ ]:
x_lst = [0.0,]*8 + [1.0,]*64


u = LowPass_FirstOrder_Bilinear()
v = LowPass_SimpleRecursive()
for x in x_lst :
    u.run(x)
    v.run(x)

In [ ]:
plt.plot(u.h['x'])
plt.plot(u.h['y'])
plt.plot(v.h['y'])
plt.grid()
plt.show()